In [7]:
pip install --upgrade llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 15.2 MB/s eta 0:00:00


In [10]:

%pip install ipykernel langchain_experimental llama-index-vector-stores-pinecone ipykernel PyMuPDF pinecone-client pypdf faiss-cpu langchain_community transformers sentence_transformers

import fitz

import os, io, json, transformers, pinecone, pypdf, faiss, sqlite3, langchain_community, langchain, openai, math, time, nltk, torch, huggingface_hub

from openai import OpenAI

from nltk.tokenize import sent_tokenize

from transformers import pipeline

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

from transformers import BertTokenizer, BertModel

import pandas as pd
import numpy as np
from io import StringIO
from dotenv import load_dotenv
from operator import itemgetter

from langchain import document_loaders, embeddings

from langchain.vectorstores import FAISS
from llama_index.vector_stores.pinecone import PineconeVectorStore

#from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.openai import OpenAIEmbedding

from langchain_experimental.text_splitter import SemanticChunker

from langchain_community.vectorstores import FAISS

from llama_index.core.schema import TextNode

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from llama_index.core.node_parser import SentenceSplitter

#from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

from pinecone import Pinecone, ServerlessSpec, Pinecone         # vector store

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)

from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

from sentence_transformers import SentenceTransformer
from sentence_transformers import util

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [70]:
load_dotenv()

openai = os.getenv('OPENAI_API_KEY')
pinecone_api_key =os.getenv('PINECONE_API_KEY')
environment =os.getenv('PINECONE_ENV')
HF_TOKEN = os.getenv('HF_TOKEN')
pc = Pinecone(api_key=pinecone_api_key)

PineconeConfigurationError: You haven't specified an Api-Key.

In [13]:
import json, os, io, re, requests, fitz
import requests
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_experimental.text_splitter import SemanticChunker
#from langchain_openai.embeddings import OpenAIEmbeddings

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.schema import TextNode
from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    KeywordExtractor,
    SummaryExtractor,

)


from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#doc = fitz.open(r"C:\GenAIhw3\product_info_pdf.pdf")
doc = fitz.open(r"/content/product_info_pdf.pdf")

In [16]:
from langchain_experimental.text_splitter import SemanticChunker

text_parser = SentenceSplitter(
    chunk_size=1024
)

In [17]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [18]:
text_chunks = []
doc_idxs = []


for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [19]:
nodes = []

for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc_idx = doc_idxs[idx]
    src_page = doc[src_doc_idx]
    nodes.append(node)

### **Create the vector store using chosen similarity metrics:**

In [25]:
use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

if use_serverless:
    spec = pinecone.ServerlessSpec(cloud='aws', region='us-east-1')
else:
    spec = pinecone.PodSpec(environment=environment)

index_name = "hw05"

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [27]:
dimensions = 1536

pc.create_index(
    name=index_name,
    dimension=dimensions,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

### **choose a similarity metric to use for the vector store:**

In [28]:
for index in pc.list_indexes():
    print(index['name'])

pc.describe_index("hw05")

pc_index = pc.Index(index_name)

vector_store = PineconeVectorStore(pinecone_index=pc_index)

hw05
cmu-handbook-alternative
cmu-handbook-recursive
cmu-handbook-semantic
hw02


In [29]:

pc_index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [40]:
llm = OpenAI(api_key=openai_key, model="gpt-3.5-turbo")
extractors = [
    KeywordExtractor(keywords=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
    SummaryExtractor(llm=llm),
]

pipeline = IngestionPipeline(
    transformations=extractors,
)
nodes = await pipeline.arun(nodes=nodes, in_place=False)



  0%|          | 0/37 [00:00<?, ?it/s]

  3%|▎         | 1/37 [00:00<00:18,  1.93it/s]

 14%|█▎        | 5/37 [00:00<00:05,  6.19it/s]

 19%|█▉        | 7/37 [00:01<00:03,  8.30it/s]

 24%|██▍       | 9/37 [00:01<00:03,  7.10it/s]

 30%|██▉       | 11/37 [00:02<00:07,  3.66it/s]

 41%|████      | 15/37 [00:02<00:04,  5.33it/s]

 49%|████▊     | 18/37 [00:02<00:02,  7.26it/s]

 54%|█████▍    | 20/37 [00:03<00:02,  7.15it/s]

 59%|█████▉    | 22/37 [00:03<00:01,  8.32it/s]

 65%|██████▍   | 24/37 [00:03<00:01,  8.15it/s]

 70%|███████   | 26/37 [00:03<00:01,  9.49it/s]

 76%|███████▌  | 28/37 [00:04<00:01,  8.42it/s]

 81%|████████  | 30/37 [00:04<00:00, 10.00it/s]

 86%|████████▋ | 32/37 [00:04<00:00,  8.52it/s]

 92%|█████████▏| 34/37 [00:04<00:00, 10.04it/s]

100%|██████████| 37/37 [00:05<00:00,  7.34it/s]


  0%|          | 0/37 [00:00<?, ?it/s]

  3%|▎         | 1/37 [00:00<00:28,  1.27it/s]

  8%|▊         | 3/37 [00:00<00:08,  3.92it/s]

 11%|█         | 4/37 [00:01<00:10,  3.22

### ***choose an embedding model to use for the vector store:**

#### **OpenAI Embeddings**

In [34]:
model_ada="text-embedding-ada-002"
small_txt_embedmodel_="text-embedding-3-small"


In [42]:
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=openai_key)

for node in nodes:
    actual_text = node.get_content(metadata_mode="all")

    node_embedding = embed_model.get_text_embedding(actual_text)

    node.embedding = node_embedding

    node.metadata["text_content"] = actual_text

### **load the embeddings into the vector store (e.g. create a vector store):**

In [43]:
vector_store.add(nodes)

Upserted vectors:   0%|          | 0/37 [00:00<?, ?it/s]

['d373d7dd-3467-4a1d-ab17-67f3dc090078',
 '2404cdb6-9799-4fd9-91aa-3c13c51688d8',
 'effa29cc-1fb2-4fde-aeeb-d212ccc78ab4',
 '250eab60-9109-4afe-a2d3-b39784bccf75',
 'bae3dca9-f82a-4bbd-9d12-bb2c4aa741cc',
 '7a4336f5-a8aa-4b9f-9e4f-918f06facca0',
 '22a2c44a-eabc-47f6-8b10-d1ad8d627183',
 '50388681-d7c6-4f12-91d4-b0b1ab635cd4',
 '88b28227-bf37-4ba8-93b3-95f2360a0d19',
 'e6b0a8d4-a6fe-43cf-b210-d64aa4f7a2bd',
 'f3d45136-8626-479e-9d9f-36d39c080a8c',
 '97c6f512-0217-4277-80f3-545ef5e4b1f7',
 '30e4747a-2855-4813-95b3-006803f290fd',
 '8a5f77a3-ec7b-464c-9d56-7f86d136764b',
 '2a0a2e88-ea1c-4287-9612-05da99d67bc0',
 '1b4e58e0-2001-45c5-bc92-79f44046843f',
 'dfb059fb-b8c7-492c-b002-67ceb00f783d',
 '9c2c9f2f-b8b1-45cd-83f0-4e2fce874eed',
 'f1f79504-0db6-4517-b85b-394edfcbf4b7',
 '53fb8332-294b-4fec-8743-a91a3ee17b79',
 '4c5d5199-db9a-4ba6-b118-ac0d6d8dd9e6',
 '71f36167-8b6f-49fe-a7c4-33c27d4ba627',
 'b9b488bc-22a7-46c1-9bbf-709864b0f14e',
 'f0854b05-b3cb-4f14-b7a7-e0ceff519c31',
 '1996b68b-d788-

In [44]:
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [45]:
print(nodes[0].metadata)

{'excerpt_keywords': 'Breville, Espresso Machine, Specialty Coffee, Grinder, Latte Art', 'questions_this_excerpt_can_answer': '1. How does the Breville BES870XL Espresso Machine contribute to the third wave specialty coffee experience at home?\n2. What specific features of the Breville Barista Express make it stand out as an all-in-one espresso machine with an integrated grinder?\n3. What accessories are included with the Breville BES870XL Espresso Machine, and how do they enhance the overall coffee-making experience for users?', 'section_summary': 'The key topics of the section include the Breville BES870XL Espresso Machine, its features such as dose control grinding, optimal water pressure, precise espresso extraction, and manual microfoam milk texturing for latte art. The section also mentions the included accessories, capacity, settings, and warranty of the machine. Additionally, it highlights how the Breville Barista Express contributes to the third wave specialty coffee experienc

In [46]:
print(nodes[0])

Node ID: d373d7dd-3467-4a1d-ab17-67f3dc090078
Text: 94-844: Generative AI Labs    Product Information    1.
Category: Home and Kitchen    We chose this category because it is the
most popular, as well as the highest  contributing category on Amazon.
Product: Breville BES870XL Espresso Machine, One Size, Brushed
Stainless Steel    Description:  About this item  The Breville Barista
Express deli...


### **Retrieve Content from the Vector Store**

In [48]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

## General Prompt & Result

#### Coffee Machine

#### Meta Quest

In [49]:
query1 = "Find information specifically about Meta Quest only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me a more wholistic understanding (description of its look, feature, and possible failure) of Meta Quest 3"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
The Meta Quest 3 is a virtual and augmented reality headset that offers an elevated gaming experience. It sports an enhanced room scanning tool that uses multiple cameras to build virtual spaces. This enables users to not only avoid physical obstacles in their play area but also interact with digital counterparts of real-world objects. Additionally, it allows streaming games from a gaming PC to the headset, which can be a boon for users with vision issues since it allows the screen to be the size of a movie theater.

The device also boasts improved resolution, lessening the "screen door" effect seen in previous iterations like the Quest 1. On the downside, it is reported to use up its battery quickly, especially when using Augmented Reality features, so keep a charging cable close by.

The Quest 3 comes with a built-in strap which some users find uncomfortable, recommending a third-party strap instead for a better fit. It should be noted that the Quest Link software, whic

#### Handpan Drum

In [50]:
query1 = "Find information specifically about Handpan Drum only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=10,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me a more wholistic understanding (description of its look, feature, and possible failure) of Handpan Drum"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
The Handpan Drum is a unique musical instrument appreciated for its outstanding sound quality and beautiful tones. A standard Handpan Drum includes a hollow, pan-like body constructed from durable materials such as steel, and multiple tone fields each tuned to specific notes. 

In terms of looks, the drum has a distinct UFO-like shape. The top ("Ding") side has a center note hammered into it and usually around seven to nine 'tone fields'. The bottom ("Gu") side often contains a round opening, producing a resonant bass sound when struck.

The key features of the instrument include its easy-to-play nature, its calming and ethereal sound, and the note arrangement which allows one to create beautiful melodies simply by tapping rhythmically with the hands. Different manufacturers may include accessories such as carrying cases, stands, and drum sticks.

However, possible failures could involve issues with tuning or sound quality over time. Some user reviews mentioned that some 

## Visual Information & Result

#### Coffee Machine

In [51]:
query1 = "Find information specifically about Breville espresso machiness only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me some bullet points of visual informations of Breville espresso machiness \
                  that helps generating product image"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
The Breville espresso machine features several visual qualities:

- Design: The machine is carefully crafted with thoughtful details and high-quality materials. Its sleek and sturdy design is appealing. Notable features include a warm storage surface for the portafilter and coffee cup, a hidden accessories tray, and a magnetized hole for the tamper.

- Performance: Breville espresso machine products are built for consistency. They are designed to grind, dose, and extract all in one, enabling you to go from beans to espresso within a minute. The controls of the machine are easy to use. 

- Accessories: The machine comes along with a variety of accessories such as a razor dose trimming tool, portafilter, filter baskets, coffee scoop, integrated tamper, stainless steel milk jug, cleaning disc, tablets, brush tool and Allen key, water filter and filter holder. 

- Built-In Grinder: The Breville Barista Express includes an integrated precision conical burr grinder, which grind

In [ ]:
query1 = "Find information specifically about Breville espresso machiness only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Give me a wholistic understanding of Breville espresso machines"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


In [57]:
query1 = "Retrieve detailed and relevant information about the design, functionality, and user experience of Breville espresso machines."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "You are an expert assistant specializing in providing insights and summaries based on the provided information."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"Here is the relevant information you can use to address the query:\n\n{retrieved_context}"},
        {"role": "user", "content": "Can you provide a detailed, comprehensive response to the question?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide a 5 line short summary of Breville espresso machines, highlighting key features, design, \
                  and user benefits which are only useful for image generation"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Breville Espresso Machines are meticulously designed, high-grade brewing appliances that are praised for delivering quality espresso consistently. One of the key features that stands out is their in-built grinding mechanism, which allows freshly ground coffee beans to be used, thereby enhancing the flavor and richness of the brewed espresso. 

The machines are compact, making them an easy fit in various kitchen setups without occupying excessive space. Their exteriors and interior displays are convenient, stylized, and pleasing to the eye, indicating that Breville values both form and function. Notably, Breville espresso machines are equipped with precise temperature control (PID) for optimal espresso extraction and a steam wand for manual microfoam milk texturing for latte art. 

In terms of user benefits, these espresso machines are lauded for their user-friendly controls and ease of use. Breville offers enough customization options, including grind size adjustment and 

In [58]:
query1 = "Retrieve detailed and relevant information about the design, functionality, and user experience of Breville espresso machines."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "You are an expert assistant specializing in providing insights and summaries based on the provided information."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"Here is the relevant information you can use to address the query:\n\n{retrieved_context}"},
        {"role": "user", "content": "Can you provide a detailed, comprehensive response to the question?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Summarize Breville espresso machines based on user reviews, highlighting visual aspects \
                  like design, material finish, steam wand, digital display, and how it fits into kitchen \
                  spaces."
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Based on numerous user reviews, majority of users commend the design, functionality, and durability of Breville espresso machines. Many users assert that Breville machines have a sleek, modern aesthetic with a robust stainless steel finish, giving them a high-end look that fits well in a variety of kitchen spaces. They also appreciate the compact size of the machines, which doesn't dominate countertop space, yet maintains a distinctive presence.

The Breville model discussed, known as the Barista Express, houses a variety of features such as the steam wand, which many users have praised for its richness and creaminess in producing milk foam. They consider this factor highly important for making lattes and cappuccinos. Another lauded aspect is the digital temperature control (PID), which ensures more precise water temperature, contributing to optimal espresso extraction.

These espresso machines also come with an integrated grinder - a feature that users highly appreciate.

In [63]:
query1 = "Retrieve detailed and relevant information about the design, functionality, and user experience of Breville espresso machines."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "You are an expert assistant specializing in providing insights and summaries based on the provided information."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"Here is the relevant information you can use to address the query:\n\n{retrieved_context}"},
        {"role": "user", "content": "Can you provide a detailed, comprehensive response to the question?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Give description of Breville espresso machines highlighting physical appearance \
                   and only visual aspects of user reviews in less than 20 words "
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Breville espresso machines showcase a sleek, stainless steel design, seamlessly blending in any kitchen setting. User reviews often mention their high-quality construction, ease of use, and consistent espresso production.


In [65]:
query1 = "Retrieve detailed and relevant information about the design, functionality, and user experience of Breville espresso machines."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "You are an expert assistant specializing in providing insights and summaries based on the provided information."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"Here is the relevant information you can use to address the query:\n\n{retrieved_context}"},
        {"role": "user", "content": "Can you provide a detailed, comprehensive response to the question?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Give description of Breville espresso machines highlighting physical appearance \
                   and only visual aspects of user reviews in less than 30 words "
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Breville espresso machines are compact, sleek in stainless steel design, and fit comfortably in any kitchen. They feature an integrated grinder and intuitive controls. User reviews praise its consistent performance, ease of use, durability, and customization options. Their attractive design and high-quality construction are frequently mentioned as highlights.


In [64]:
query1 = "Retrieve detailed and relevant information about the design, functionality, and user experience of Breville espresso machines."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "You are an expert assistant specializing in providing insights and summaries based on the provided information."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"Here is the relevant information you can use to address the query:\n\n{retrieved_context}"},
        {"role": "user", "content": "Can you provide a detailed, comprehensive response to the question?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Give description of Breville espresso machines highlighting physical appearance \
                   and only visual aspects of user reviews(positive and negative) in less than 30 words "
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Breville espresso machines stand out with their sleek, stainless steel design, offering strength and ease of use. They are compact, capable of fitting into any kitchen setup, and feature various intuitive controls. User reviews note the consistency of the espresso produced, the quick heating, and easy-to-adjust grind levels. However, some users highlight that getting the grind level just right can be challenging and that the machine may run out of water during operation. Moreover, potential premature failure of the grinder is a concern.


In [69]:
query1 = "Retrieve detailed and relevant information about the design, functionality, and user experience of Breville espresso machines."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "You are assisting a 3D artist to generate an image. Give more details on how to get images"},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"Here is the relevant information you can use to address the query:\n\n{retrieved_context}"},
        {"role": "user", "content": "Can you provide a detailed, comprehensive response to the question?"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Give description of Breville espresso machines highlighting physical appearance \
                   and only visual aspects of user reviews(positive and negative) in less than 30 words "
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Breville espresso machines are robust and elegantly designed products known for their high-quality stainless steel built. Users praise them for consistent espresso production, user-friendly controls, and thoughtful design elements. Some users note high durability, easy usage, and impressive performance. A customized coarseness of grinds, manual micro-foam milk texturing, and easy dosage control are highly acclaimed features. However, few users indicate problems with grinder maintenance and concerns about premature failure. In summary, users find Breville espresso machines reliable and worthwhile for the exquisite coffee-making experience they provide.
